# Challenge 1: Analyze Specific Products With Regex

Machine Learning & APIs with Python Course (DS4B 201-P)

Business Science

# Challenge Summary

**The Situation:** Your boss comes back and asks you to modify the analysis to understand customer's attributes that are interested in purchasing learning labs vs the R-Track Courses. You agree since there could be differences or similarities between the two populations. But how would you update the analysis?

**Your Proposed Solution:** You recommend to update the analysis so different products can be targeted such as "All Products" or lists of product names (e.g. "Learning Labs PRO"). You have heard that Regular Expression can help to filter, and you propose updating the Targeting function `els.db_read_els_data_regex()` to permit filtering products with regular expressions. 

## Objectives:

1. Update `email_lead_scoring` functionality to filter the products

2. Apply the existing functionality to analyze trends and compare purchasers of products.

## Getting Started

To read in the data, make sure your current working directory is set to the project directory. Two useful jupyter magic commands are:

1. `%pwd`: Print working directory (you can detect your current directory)
2. `%cd`: You can change directory to your working directory using relative paths or full paths.

In [1]:
%pwd

'c:\\Users\\natal\\OneDrive\\Desktop\\Business Science\\ds4b_201p_course\\ds4b_201p_course'

In [2]:
%cd c:\\Users\\natal\\OneDrive\\Desktop\\Business Science\\ds4b_201p_course\\ds4b_201p_course

c:\Users\natal\OneDrive\Desktop\Business Science\ds4b_201p_course\ds4b_201p_course


In [3]:
%pwd

'c:\\Users\\natal\\OneDrive\\Desktop\\Business Science\\ds4b_201p_course\\ds4b_201p_course'

## Python Package Imports

In [4]:
import pandas as pd
import numpy as np
import sqlalchemy as sql
import sweetviz as sv

import email_lead_scoring as els

# Part 1: Using Regex to Locate Products in the Products Table

### Step 1: List the Database Table Names

Use `db_read_els_table_names()` to get the tables in the database. 

In [5]:
els.db_read_els_table_names()

['Products', 'Subscribers', 'Tags', 'Transactions', 'Website']

### Step 2: Collect the Products Table

1. Create a `products_df` by reading in the raw els table "Products"
2. Convert the type of "product_id" to `int`.
3. Display the first 10 rows with the `.head()` method

In [7]:
product_df = els.db_read_raw_els_table(table='Products')
product_df['product_id'] = product_df['product_id'].astype(int)
product_df.head(10)

product_id                                        description  \
0           1  4-Course Bundle - Machine Learning + Expert We...   
1           2  4-Course Bundle - Machine Learning + Expert We...   
2           3  4-Course Bundle - Machine Learning + Expert We...   
3           4  5 Course Bundle - Machine Learning + Web Apps ...   
4           5  5 Course Bundle - Machine Learning + Web Apps ...   
5           6  5 Course Bundle - Machine Learning + Web Apps ...   
6           7  5 Course Bundle - Machine Learning + Web Apps ...   
7           8  Bundle - Data Science Starter Kit: R-Track - C...   
8           9  Bundle - Data Science Starter Kit: R-Track - C...   
9          10  Bundle - DS For Business + Web Apps (Level 1):...   

   suggested_price  
0       241.666667  
1       441.666667  
2      2500.000000  
3       275.000000  
4       194.444444  
5       525.000000  
6      3000.000000  
7       216.666667  
8      1200.000000  
9       145.833333

Next, get the product descriptions as a pandas Series. And store these as `product_descriptions_series`. 

In [9]:
product_description_series = product_df['description']


### Step 3: Use Regex To Filter Data

In this next part, we will use `regex`, a text matching algorithm, to filter the product IDs that we wish to analyze. 

* Use `.str.match("Learning Lab")` to detect the presence of learning lab. 
* Store the resulting pandas series as `_filter`

In [11]:
_filter = product_description_series.str.match('Learning Lab')

Use the `_filter` to select product ID's for product descriptions that contain "Learning Lab"

* Take the `products_df` table
* Use `[_filter]` to mask the rows that are `False`
* Select the `['product_id']` that correspond to the masked rows
* Store the result as `product_ids_filtered_series`

In [13]:
product_ids_filtered_series = product_df[_filter]['product_id']
product_ids_filtered_series

36    39
37    40
38    41
39    42
40    43
41    44
42    45
43    46
Name: product_id, dtype: int32

# Part 2: Filtering the Transactions Table by Product ID

### Step 1: Filter the Transactions Corresponding to the Filtered Product IDs

Next, we'll filter the transactions table to down to just the transactions with the product IDs we are interested in. 

* Read the raw "Transactions" Table, and store it as `transactions_df` 
* Convert the "product_id" column to type `int`
* Store the result as `transactions_df`

In [14]:
transactions_df = els.db_read_raw_els_table(table='Transactions')
transactions_df['product_id'] = transactions_df['product_id'].astype(int)
transactions_df

transaction_id purchased_at          user_full_name  \
0                  1   2018-04-29           Deliah Kemmer   
1                  2   2018-04-29   Dr. Ronal Keeling DVM   
2                  3   2018-04-29       Taya Ondricka-Orn   
3                  4   2018-04-29         Camren Auer DDS   
4                  5   2018-04-29   Dr Janna Glover Ph.D.   
...              ...          ...                     ...   
4657            4658   2020-12-31         Bedford Farrell   
4658            4659   2020-12-31  Michelina Pfannerstill   
4659            4660   2020-12-31     Mrs. Irene Feil PhD   
4660            4661   2020-12-31          Conner Carroll   
4661            4662   2020-12-31        Francis Mosciski   

                            user_email charge_country  product_id  
0              deliah.kemmer@gmail.com             US          31  
1       dr.ronal.keeling.dvm@gmail.com             NZ          31  
2         taya.ondrickaorn@hotmail.com             US          31  
3             camren.auer.dds@live.com             AU          31  
4        dr.janna.glover.phd@gmail.com             GB          31  
...                                ...            ...         ...  
4657         bedford.farrell@yahoo.com             US          41  
4658  michelina.pfannerstill@gmail.com             IN          41  
4659      mrs.irene.feil.phd@gmail.com             US          23  
4660          conner.carroll@gmail.com             AU           1  
4661        francis.mosciski@gmail.com             IN          37  

[4662 rows x 6 columns]

Next, create `_filter_2` :

* Filter `transaction_df['product_id']` which are in the `product_ids_filtered_series`. 
* The result is a pandas series of Boolean True/False.
* Store this output as `_filter_2`.

In [15]:
_filter_2 = transactions_df['product_id'].isin(product_ids_filtered_series)
_filter_2

0       False
1       False
2       False
3       False
4       False
        ...  
4657     True
4658     True
4659    False
4660    False
4661    False
Name: product_id, Length: 4662, dtype: bool

Use `_filter_2` to mask the `transactions_df` so only the rows containing "Learning Labs" products are shown. 
Store this result as `transactions_filtered_df`.

In [17]:
transactions_filtered_df = transactions_df[_filter_2]
transactions_filtered_df

transaction_id purchased_at              user_full_name  \
258              259   2019-04-11            Josephus Goldner   
259              260   2019-04-11               Marion Corwin   
260              261   2019-04-11     Dr. Kash Schroeder m.d.   
262              263   2019-04-13                Harrold Kris   
263              264   2019-04-15           Taya Ondricka-Orn   
...              ...          ...                         ...   
4652            4653   2020-12-30  Miss. Kaylen Gleichner dvm   
4655            4656   2020-12-31                 Rita Carter   
4656            4657   2020-12-31             Dr. Liston Cole   
4657            4658   2020-12-31             Bedford Farrell   
4658            4659   2020-12-31      Michelina Pfannerstill   

                               user_email charge_country  product_id  
258            josephus.goldner@gmail.com             BE          46  
259               marion.corwin@gmail.com             IE          46  
260        dr.kash.schroeder.md@gmail.com             GB          46  
262                harrold.kris@gmail.com             US          46  
263          taya.ondrickaorn@hotmail.com             US          46  
...                                   ...            ...         ...  
4652  miss.kaylen.gleichner.dvm@gmail.com             HR          44  
4655                rita.carter@gmail.com             IN          41  
4656             dr.liston.cole@gmail.com             US          41  
4657            bedford.farrell@yahoo.com             US          41  
4658     michelina.pfannerstill@gmail.com             IN          41  

[2669 rows x 6 columns]

# Part 3: Update the Email Lead Scoring Function to Add A Regex Command

Use what you just learned to make a new function called: `db_read_els_data_regex()`.

### Instructions:

* Begin with the `db_read_els_data()` function shown below, but has been renamed to `db_read_els_data_regex()`. 
  * A new argument has been added called `transaction_regex = "All Products"` which will be used to filter transactions by product matching regex. 
* You will:
  * Add the Products Table into the `engine.connect()` script
  * Filter the products table only when `transaction_regex != "All Products"`
  * Filter the transactions table using the product IDs from the filtered products table only when `transaction_regex != "All Products"`

In [ ]:
### Update this function:

def db_read_els_data_regex(
    transaction_regex = "All Products",
    conn_string = "sqlite:///00_database/crm_database.sqlite"
):
        
    # Connect to engine
    engine = sql.create_engine(conn_string)
    
    # Raw Data Collect
    with engine.connect() as conn:
        
        # Subscribers        
        subscribers_df = pd.read_sql("SELECT * FROM Subscribers", conn)
        
        subscribers_df['mailchimp_id'] = subscribers_df['mailchimp_id'].astype('int')

        subscribers_df['member_rating'] = subscribers_df['member_rating'].astype('int')

        subscribers_df['optin_time'] = subscribers_df['optin_time'].astype('datetime64')

        # Tags
        tags_df = pd.read_sql("SELECT * FROM Tags", conn)
        
        tags_df['mailchimp_id'] = tags_df['mailchimp_id'].astype("int")
        
        # Transactions
        transactions_df = pd.read_sql("SELECT * FROM Transactions", conn)
        
        transactions_df['purchased_at'] = transactions_df['purchased_at'].astype('datetime64')

        transactions_df['product_id'] = transactions_df['product_id'].astype('int')
        
    # MERGE TAG COUNTS
    
    user_events_df = tags_df \
        .groupby('mailchimp_id') \
        .agg(dict(tag = 'count')) \
        .set_axis(['tag_count'], axis=1) \
        .reset_index()
    
    subscribers_joined_df = subscribers_df \
        .merge(user_events_df, how='left') \
        .fillna(dict(tag_count = 0))
        
    subscribers_joined_df['tag_count'] = subscribers_joined_df['tag_count'].astype('int')
    
    # MERGE TARGET VARIABLE
    emails_made_purchase = transactions_df['user_email'].unique()
    
    subscribers_joined_df['made_purchase'] = subscribers_joined_df['user_email'] \
        .isin(emails_made_purchase) \
        .astype('int')
    
        
    return subscribers_joined_df

### Solution:

In [19]:

# Final Solution:
def db_read_els_data_regex(
    transaction_regex = "All Products",
    conn_string = "sqlite:///00_database/crm_database.sqlite"
):
        
    # Connect to engine
    engine = sql.create_engine(conn_string)
    
    # Raw Data Collect
    with engine.connect() as conn:
        
        # Subscribers        
        subscribers_df = pd.read_sql("SELECT * FROM Subscribers", conn)
        
        subscribers_df['mailchimp_id'] = subscribers_df['mailchimp_id'].astype('int')

        subscribers_df['member_rating'] = subscribers_df['member_rating'].astype('int')

        subscribers_df['optin_time'] = subscribers_df['optin_time'].astype('datetime64')

        # Tags
        tags_df = pd.read_sql("SELECT * FROM Tags", conn)
        
        tags_df['mailchimp_id'] = tags_df['mailchimp_id'].astype("int")
        
        # Transactions
        transactions_df = pd.read_sql("SELECT * FROM Transactions", conn)
        
        transactions_df['purchased_at'] = transactions_df['purchased_at'].astype('datetime64')

        transactions_df['product_id'] = transactions_df['product_id'].astype('int')
        
        # Products
        if transaction_regex != 'All Products':
            product_df = pd.read_sql('Select * FROM Products', conn)
            product_df['product_id'] = product_df['product_id'].astype(int)
            
            product_description_series = product_df['description']
            _filter = product_description_series.str.match(transaction_regex)
            product_ids_filtered_series = product_df[_filter]['product_id']
            
            _filter_2 = transactions_df['product_id'].isin(product_ids_filtered_series)
            transactions_df = transactions_df[_filter_2]
        
    # MERGE TAG COUNTS
    
    user_events_df = tags_df \
        .groupby('mailchimp_id') \
        .agg(dict(tag = 'count')) \
        .set_axis(['tag_count'], axis=1) \
        .reset_index()
    
    subscribers_joined_df = subscribers_df \
        .merge(user_events_df, how='left') \
        .fillna(dict(tag_count = 0))
        
    subscribers_joined_df['tag_count'] = subscribers_joined_df['tag_count'].astype('int')
    
    # MERGE TARGET VARIABLE
    emails_made_purchase = transactions_df['user_email'].unique()
    
    subscribers_joined_df['made_purchase'] = subscribers_joined_df['user_email'] \
        .isin(emails_made_purchase) \
        .astype('int')
    
        
    return subscribers_joined_df

## BONUS: Sweetviz Report

Next, we are going to tackle making a Sweetviz Report to identify any interesting insights

### Step 1: Use the `db_read_els_data_regex()` function to identify Learning Lab transactions

* Use the `db_read_els_data_regex()` function to indentify transactions that match "Learning Lab"
* Store the result as `subscribers_learning_lab_purchase_flag_df`

In [20]:
subscribers_learning_lab_purchase_flag_df = db_read_els_data_regex( transaction_regex = "Learning Lab")
subscribers_learning_lab_purchase_flag_df

mailchimp_id          user_full_name                       user_email  \
0                 3       Garrick Langworth      garrick.langworth@gmail.com   
1                 4         Cordell Dickens        cordell.dickens@gmail.com   
2                 8               Inga Dach              inga.dach@gmail.com   
3                10     Ferdinand Bergstrom    ferdinand.bergstrom@gmail.com   
4                11          Justen Simonis         justen.simonis@gmail.com   
...             ...                     ...                              ...   
19914         33405   Ms. Felicity Moore MD   ms.felicity.moore.md@gmail.com   
19915         33406            Shirley Rowe           shirley.rowe@gmail.com   
19916         33407  Jarrett Walker-Carroll  jarrett.walkercarroll@gmail.com   
19917         33408            Tanja Herzog           tanja.herzog@gmail.com   
19918         33409          Farrell Murphy         farrell.murphy@gmail.com   

       member_rating optin_time country_code  tag_count  made_purchase  
0                  2 2019-05-22           in          6              0  
1                  4 2018-11-19           it          0              0  
2                  2 2018-11-19         None          0              0  
3                  2 2020-03-20           co          3              0  
4                  2 2020-04-14         None          0              0  
...              ...        ...          ...        ...            ...  
19914              2 2018-11-18           se          0              0  
19915              1 2019-03-12           br          0              0  
19916              2 2019-09-09           in          0              0  
19917              2 2019-10-24         None          2              0  
19918              2 2020-06-19           ae          0              0  

[19919 rows x 8 columns]

### Step 2: Use Sweetviz Analyze

* Create an analysis report using `sv.analyze()`
* Set the target feature to "made_purchase"
* Store the result as `report`

In [21]:
report = sv.analyze(
    subscribers_learning_lab_purchase_flag_df,
    target_feat='made_purchase'
)


                                             |          | [  0%]   00:00 -> (? left)

### Step 3: Save the report

* Use `report.show_html()` to create a report
* Store it in the file location: `"challenge_01/subscriber_learninglab_report.html"`

In [22]:
report.show_html(
    filepath = 'challenge_01/subscriber_learninglab_report.html'
)

Report challenge_01/subscriber_learninglab_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
